In [2]:
import numpy as np
import shap
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
x_test = np.load('annot_keras.npy')

In [4]:
x_mask = np.load('masks_keras.npy')

In [8]:
base = '../../hdd/Amitoj/OCT2017/annot10_inc_res_attr/'

In [9]:
"""explainers = ['shap_select', 'shap_rand', 'guided_backprop', 'deep_taylor', 'input_t_gradient', 'lrp.z', 'integrated_gradients',
              'gradient', "smoothgrad", "deconvnet", 'deeplift', 'saliency', 'occlusion', 'lrp.epsilon']"""
explainers = ["shap_rand", 'guided_backprop', 'input_t_gradient', 'lrp.epsilon','deconvnet', 'saliency', 'intgrad', 'deeplift', 'occlusion']
explainers.sort()

In [10]:
mask_base = 'OCT-annotations/masks/'
mask_dirs = [mask_base+'CNV', mask_base+'DME', mask_base+'DRUSEN']
mask_files = []
for mdir in mask_dirs:
    mfiles = [mdir + '/' + f for f in listdir(mdir) if isfile(join(mdir, f))]
    mfiles.sort()
    mask_files.append(mfiles)

In [12]:
atr_list = [] #array of all attributions -> can get massive for large dataset
for exp in explainers:
    atr_exp = []
    for m in range(1): #over all the models
        atr_exp.append(np.load(base + exp + '_model_' + str(m) + '.npy'))
    atr_list.append(atr_exp)

In [13]:
for e, exp in enumerate(explainers):
    print(e, exp)

0 deconvnet
1 deeplift
2 guided_backprop
3 input_t_gradient
4 intgrad
5 lrp.epsilon
6 occlusion
7 saliency
8 shap_rand


In [18]:
exp = 7
m = 0
for exp in range(len(explainers)):
    atr = atr_list[exp][m]
    cos_arr = []
    for cat, masks in enumerate(mask_files):
        cos_cat_arr = []
        for num, mask in enumerate(masks):
            marked = mpimg.imread(mask)
            res = resize(marked[:,:,3], (299,299))
            atr_curr = atr[cat][num:num+1]
            #obj = shap.image_plot(atr_curr, np.array([marked]) , show = False)
            #obj = shap.image_plot(atr_curr, n , show = False)
            plt.show()
            atr_flat = atr_curr.flatten()
            atr_flat[atr_flat < 0] = 0
            res_flat = res.flatten()
            cos = cosine_similarity([atr_flat], [res_flat])
            cos_cat_arr.append(cos)
        cos_arr.append(cos_cat_arr)
    print(explainers[exp], np.sum(cos_arr[0])/10, np.sum(cos_arr[1])/10, np.sum(cos_arr[2])/10, np.sum(np.array(cos_arr))/30)

deconvnet 0.35027591619667586 0.325307249236621 0.2141949069555221 0.296592690796273
deeplift 0.37013735838624606 0.25409316715928637 0.2763043759177885 0.30017830048777355
guided_backprop 0.4297425722066732 0.3888872788031329 0.36974995082358925 0.39612660061113175
input_t_gradient 0.37835117107713134 0.29489926330328753 0.27788140048580867 0.31704394495540916
intgrad 0.2885895624931447 0.2449865321656096 0.3264012909901845 0.2866591285496463
lrp.epsilon 0.36843603609341724 0.35200003060696033 0.2826665861945917 0.33436755096498977
occlusion 0.6052065972458873 0.4170132285722434 0.3400990953473296 0.4541063070551534
saliency 0.5817868008623351 0.418745885483523 0.36795569273026646 0.4561627930253748
shap_rand 0.33219955241340604 0.3208347104345632 0.3095600974384515 0.3208647867621402


In [13]:
# find dice coeff and IOU for each class
# take weighted average for overall performance as Drusen has very small area

In [14]:
# do cosine similarity after resizing the attribution to image, clip negative points zero